In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
from collections import Counter
import json
import math
import copy
import os, sys

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, f1_score
from scipy.special import softmax


### 数据预处理

In [3]:
# path = './data/rec_data/train-dataset.csv'
# # path = './data/rec_data/test-dataset.csv'
# data = pd.read_csv(path)
# data['time'] = pd.to_datetime(data['logTs'],unit='ms',origin=pd.to_datetime('1970-01-01 08:00:00'))
# data['Hour'] = data['time'].dt.hour
# data['Min'] = data['time'].dt.hour*60+data['time'].dt.minute
# data['seq'] = data['userSeq'].str.split('[;:]').fillna(0)
# data

In [4]:
# df1 = pd.read_csv('./data/rec_data/train-co-occurrence-feature.csv')
# df2 = pd.read_csv('./data/rec_data/train-itemId-feature.csv')
# df3 = pd.read_csv('./data/rec_data/train-emotion-feature.csv')

# train_all_feature = pd.concat([data,df1,df2,df3],axis=1)
# del train_all_feature['seqLen']
# train_all_feature['seqLen'] = train_all_feature['userSeq'].str.count(';').fillna(0).astype('int')
# del train_all_feature['userSeq']

# print(train_all_feature.columns)
# train_all_feature.to_csv('./data/feature/train_all_feature.csv', index=False)
# train_all_feature.head()

In [5]:
# path = './data/rec_data/test-dataset.csv'

# data = pd.read_csv(path)
# data['time'] = pd.to_datetime(data['logTs'],unit='ms',origin=pd.to_datetime('1970-01-01 08:00:00'))
# data['Hour'] = data['time'].dt.hour
# data['Min'] = data['time'].dt.hour*60+data['time'].dt.minute
# data['seq'] = data['userSeq'].str.split('[;:]').fillna(0)

# df1 = pd.read_csv('./data/rec_data/test-co-occurrence-feature.csv')
# df2 = pd.read_csv('./data/rec_data/test-itemId-feature.csv')
# df3 = pd.read_csv('./data/rec_data/test-emotion-feature.csv')

# test_all_feature = pd.concat([data,df1,df2,df3],axis=1)
# del test_all_feature['seqLen']
# test_all_feature['seqLen'] = test_all_feature['userSeq'].str.count(';').fillna(0).astype('int')
# del test_all_feature['userSeq']

# print(test_all_feature.columns)
# print(len(test_all_feature))
# test_all_feature.to_csv('./data/feature/test_all_feature.csv', index=False)
# test_all_feature

#### 数据读取

In [2]:
path = './data/feature/train_all_feature.csv'
train_feature = pd.read_csv(path)
path = './data/feature/test_all_feature.csv'
test_feature = pd.read_csv(path)
print(train_feature.columns)
print(len(train_feature)+len(test_feature))
train_feature

Index(['sampleId', 'label', 'pvId', 'suv', 'itemId', 'logTs', 'operator',
       'browserType', 'deviceType', 'osType', 'province', 'city', 'time',
       'Hour', 'Min', 'seq', 'sum', 'mean', 'std', 'nor_sum', 'nor_mean',
       'nor_std', 'posCnt', 'negCnt', 'historyCnt', 'allCnt', 'entity_cnt',
       'mean_emotion_gap', 'seqLen'],
      dtype='object')
5656022


,sampleId,label,pvId,suv,itemId,logTs,operator,browserType,deviceType,osType,...,nor_sum,nor_mean,nor_std,posCnt,negCnt,historyCnt,allCnt,entity_cnt,mean_emotion_gap,seqLen
0,1,0,1420332726637uyIZR2O,1581173072343wnwm3q,10092752,1641039890894,2203920,1583865,2326891,3585570,...,0,0,0,1076,5732,36535,6808,0,1.0,20
1,2,0,1420332726637uyIZR2O,1581173072343wnwm3q,10085565,1641039890894,2203920,1583865,2326891,3585570,...,176,8,16,1392,6463,79661,7855,5,0.0,20
2,3,0,1420332726637uyIZR2O,1581173072343wnwm3q,10105937,1641039890894,2203920,1583865,2326891,3585570,...,65,3,8,1820,9782,79034,11602,2,0.0,20
3,4,0,1420332726637uyIZR2O,1581173072343wnwm3q,10082274,1641039890894,2203920,1583865,2326891,3585570,...,102,4,13,1927,9660,47663,11587,3,0.0,20
4,5,0,1420332726637uyIZR2O,1581173072343wnwm3q,10083446,1641039890894,2203920,1583865,2326891,3585570,...,18,0,3,1607,7436,35258,9043,1,0.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233799,4233800,0,1641237780999Vpq1Hnl,1497538865583737,10119437,1641225378200,1570834,1583737,2326891,3585570,...,396,18,48,1656,5899,47535,7555,3,0.0,20
4233800,4233801,0,1641237780999Vpq1Hnl,1497538865583737,10123561,1641225390758,1570834,1583737,2326891,3585570,...,0,0,0,13,90,7122,103,0,1.0,20
4233801,4233802,0,1641237780999Vpq1Hnl,1497538865583737,10122359,1641225381972,1570834,1583737,2326891,3585570,...,0,0,0,154,903,17163,1057,0,1.0,20
4233802,4233803,0,1641237780999Vpq1Hnl,1497538865583737,10122833,1641225322020,1570834,1583737,2326891,3585570,...,0,0,0,228,1019,89892,1247,0,1.0,20


#### 生成词表

In [3]:
entity_map = {}
with open('./data/rec_data/recommend_content_entity_0317.txt') as f:
    for line in f:
        if len(line.strip()):
            js = json.loads(line)
            entity_map[int(js['id'])] = js['entity']
print(len(entity_map))

146791


In [5]:
# featureNames = ['itemId', 'province', 'city', 'Hour', 'deviceType', 
#                 'sum', 'seqLen', 'entity_cnt', 'posCnt', 'negCnt']
featureNames = [ 'province', 'city', 'deviceType', 'itemId', 'Hour', 'seqLen', 'entity_cnt']

In [94]:
def generate_vocab(featureList):
    
    featureDic = dict()
    entityCounter = Counter()
    itemCounter = Counter()
    featureDic['[sep]'] = 0
    featureDic['[noHistory]'] = 1
    idx = 2
    for i in [-2,-1,0,1,2]:
        featureDic[f'emotion_{i}'] = idx
        idx += 1
    for i in range(600):
        featureDic[f'timeGap_{i}s'] = idx
        idx += 1
    featureDic[f'timeGap_MoreThan10min'] = idx
    idx += 1
    for i in range(0,100):
        featureDic[f'timeGap_{i}h'] = idx
        idx += 1
    featureDic[f'timeGap_MoreThan100h'] = idx
    idx += 1
    featureDic[f'timeGap_LessThan0h'] = idx
    idx += 1
    feature = pd.concat(featureList,axis=0)
    print(f'训练集+测试集总长={len(feature)}')
    print(feature.nunique())
    
    for name in featureNames:
        if name=='itemId': 
            for itemId in feature[name].unique():
                itemCounter[itemId] += 1
                for entity in entity_map[itemId]:
                    entityCounter[entity] += 1
        else:
            for each in feature[name].unique():
                featureDic[f'{name}_{each}'] = idx
                idx += 1
    for seqArr in tqdm(feature['seq']):
        if len(seqArr) == 1:  # 无历史信息
            continue
        seqArr = json.loads(seqArr.replace('\'', '\"'))
        assert len(seqArr)%2 == 0
        for i in range(0,len(seqArr),2):
            time = int(seqArr[i+1])
            itemId = int(seqArr[i])
            itemCounter[itemId] += 1
            for entity in entity_map[itemId]:
                entityCounter[entity] += 1
    
    print(f'item词典大小={len(itemCounter)}')
    for i,itemId in enumerate(itemCounter):
        featureDic[f'itemId_{itemId}'] = idx
        idx += 1
    
    print(f'实体词典大小={len(entityCounter)}')
    for i,entity in enumerate(entityCounter):
        assert entity not in featureDic
        featureDic[entity] = idx
        idx += 1
        
    print(f'特征词典总大小={len(featureDic)}') 
    return featureDic
#     print(featureDic)
#     print(len(entityCounter),entityCounter) 
featureDic = generate_vocab([train_feature, test_feature])
with open('./data/feature/featureDic.json', 'w') as f:
    json.dump(featureDic,f)

训练集+测试集总长=5656022
sampleId            4233804
label                     2
pvId                 400031
suv                  354976
itemId                 3319
logTs               2816572
operator                469
browserType              31
deviceType                4
osType                   12
province                 36
city                    337
time                2816572
Hour                     24
Min                    1440
seq                  329228
sum                     125
mean                     80
std                      76
nor_sum               15024
nor_mean               3566
nor_std                6490
posCnt                  499
negCnt                  761
historyCnt             1573
allCnt                 1056
entity_cnt               24
mean_emotion_gap      19574
seqLen                   21
testSampleId        1422218
dtype: int64


100%|██████████| 5656022/5656022 [04:02<00:00, 23308.41it/s]


item词典大小=107835
实体词典大小=241319
特征词典总大小=350310


#### group时间间隔统计

In [89]:
feature_group = train_feature.groupby('pvId')
timeGap = []
historyGap = []
for group in tqdm(feature_group):
    firstTime = group_df['logTs'].min()
    timeGap.append(int((group_df['logTs'].max()-firstTime)/1000))
    for seqArr in group[1]['seq']:
        if len(seqArr) == 1:  # 无历史信息
            continue
        seqArr = json.loads(seqArr.replace('\'', '\"'))
        assert len(seqArr)%2 == 0
        for i in range(0,len(seqArr),2):
            time = int(seqArr[i+1])
#             assert firstTime > time
            historyGap.append(int((firstTime - time)/1000/3600))
timeGap = pd.Series(timeGap)
timeGap.describe()

100%|██████████| 299919/299919 [02:16<00:00, 2199.21it/s]


count    299919.0
mean        587.0
std           0.0
min         587.0
25%         587.0
50%         587.0
75%         587.0
max         587.0
dtype: float64

In [90]:
historyGap = pd.Series(historyGap)
historyGap.describe()

count    6.314284e+07
mean     3.921326e+01
std      4.413392e+01
min     -2.600000e+01
25%      8.000000e+00
50%      3.200000e+01
75%      5.800000e+01
max      1.621000e+03
dtype: float64

In [ ]:
for i in range(0,120,24):
    print(f'历史浏览时间间隔小于{i}h\t'len(historyGap[historyGap<=i])/len(historyGap))


#### 生成训练数据

In [9]:
with open('./data/feature/featureDic.json') as f:
    featureDic = json.load(f)

feature_group = train_feature.groupby('pvId')
lengths = []
train_df = []
for group in tqdm(feature_group):
    group_df = group[1].sort_values(by = 'logTs')  # 按时间顺序排序
#     group_df['time'] = pd.to_datetime(group_df['logTs'],unit='ms',origin=pd.to_datetime('1970-01-01 08:00:00'))
#     print(group_df[featureNames+['logTs', 'time']])
    
#     userFeatures = ['province', 'city', 'deviceType', 'seqLen']
#     for each in userFeatures:
#         assert len(group_df[each].unique()==1)
        
    x, y = [], []
    '''
    user 特征 (只添加一次)
    '''
    for item in group_df[featureNames].itertuples():
        x.append(featureDic['[sep]'])
        x.append(featureDic[f'province_{item.province}'])
        x.append(featureDic[f'city_{item.city}'])
        x.append(featureDic[f'deviceType_{item.deviceType}'])
        x.append(featureDic[f'seqLen_{item.seqLen}'])           # 历史记录长度
        x.append(featureDic['[sep]'])        
        
        y.extend([2] * len(x))
        break
    firstTime = 0
    
    '''
    文章特征
    '''
    for idx, item in enumerate(group_df[featureNames+['label', 'logTs']].itertuples()):
        if idx == 0:
            firstTime = int(item.logTs)
        x.append(featureDic[f'itemId_{item.itemId}'])
        x.append(featureDic[f'Hour_{item.Hour}'])
        x.append(featureDic[f'entity_cnt_{item.entity_cnt}'])   # 共现实体数（加权）
        
        gap = int(abs(int(item.logTs)-firstTime)/1000) # 距离第一篇文章时间间隔（秒）
        if gap <= 599:
            x.append(featureDic[f'timeGap_{gap}s'])
        else:
            x.append(featureDic['timeGap_MoreThan10min'])
        x.append(featureDic['[sep]'])
        
        y.extend([item.label]*5)
    assert len(x) == len(y)
#     print(len(x))
    '''
    历史特征 (只添加一次)
    '''
    seqArr = group_df['seq'].values[0]
    if len(seqArr) == 1:  # 无历史信息
        x.append(featureDic['[noHistory]'])
        x.append(featureDic['[sep]'])
        y.extend([3, 3])
        continue
    seqArr = json.loads(seqArr.replace('\'', '\"'))
    assert len(seqArr)%2 == 0
    historyPair = []
    for i in range(0,len(seqArr),2):
        time = int(seqArr[i+1])
        itemId = int(seqArr[i])
        historyPair.append([itemId, time])
    historyPair = sorted(historyPair,key=lambda x:-x[1]) # 按时间排序离当前时间近的排前面
#         pprint(historyPair)
    for itemId, time in historyPair:
        historyTimeGap = int((firstTime - time)/1000/3600)
        if historyTimeGap<0:
            key = 'timeGap_LessThan0h'
        elif historyTimeGap>=100:
            key = 'timeGap_MoreThan100h'
        else:
            key = f'timeGap_{historyTimeGap}h'

        x.append(featureDic[f'itemId_{item.itemId}'])
        x.append(featureDic[key])
        x.append(featureDic['[sep]'])
        y.extend([3, 3, 3])
#     print(len(x), x)
#     print(len(y), y)
    assert len(x) == len(y)
    lengths.append(len(x))
    train_df.append([x,y])
#     break
train_df = pd.DataFrame(train_df)
lengths = pd.Series(lengths)
lengths.describe()

100%|██████████| 299919/299919 [10:32<00:00, 474.05it/s]


count    247082.000000
mean        130.839041
std          21.938947
min          14.000000
25%         119.000000
50%         144.000000
75%         144.000000
max        1754.000000
dtype: float64

In [10]:
train_df.columns = ['ids','labels']
train_df.to_csv('./data/transformer_train_data.csv',index=False)
train_df

,ids,labels
0,"[0, 711, 747, 1083, 1130, 0, 2863, 1103, 1132,...","[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
1,"[0, 710, 746, 1083, 1111, 0, 1156, 1087, 1132,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 711, 747, 1083, 1112, 0, 1171, 1088, 1136,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, ..."
3,"[0, 711, 747, 1083, 1111, 0, 2103, 1098, 1135,...","[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
4,"[0, 711, 747, 1083, 1111, 0, 1194, 1090, 1133,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
247077,"[0, 717, 785, 1083, 1111, 0, 2139, 1110, 1132,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
247078,"[0, 723, 992, 1083, 1111, 0, 3435, 1110, 1132,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
247079,"[0, 711, 747, 1083, 1125, 0, 2139, 1110, 1132,...","[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
247080,"[0, 720, 757, 1083, 1111, 0, 1187, 1107, 1133,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


#### 生成测试集

In [112]:
feature_group = test_feature.groupby('pvId')
lengths = []
test_df = []
for group in tqdm(feature_group):
    group_df = group[1].sort_values(by = 'logTs')  # 按时间顺序排序
#     group_df['time'] = pd.to_datetime(group_df['logTs'],unit='ms',origin=pd.to_datetime('1970-01-01 08:00:00'))
#     print(group_df[featureNames+['logTs', 'time']])
    
#     userFeatures = ['province', 'city', 'deviceType', 'seqLen']
#     for each in userFeatures:
#         assert len(group_df[each].unique()==1)
        
    x = []
    '''
    user 特征 (只添加一次)
    '''
    for item in group_df[featureNames].itertuples():
        x.append(featureDic['[sep]'])
        x.append(featureDic[f'province_{item.province}'])
        x.append(featureDic[f'city_{item.city}'])
        x.append(featureDic[f'deviceType_{item.deviceType}'])
        x.append(featureDic[f'seqLen_{item.seqLen}'])           # 历史记录长度
        x.append(featureDic['[sep]'])        
        
        break
    firstTime = 0
    
    '''
    文章特征
    '''
    for idx, item in enumerate(group_df[featureNames+['logTs']].itertuples()):
        if idx == 0:
            firstTime = int(item.logTs)
        x.append(featureDic[f'itemId_{item.itemId}'])
        x.append(featureDic[f'Hour_{item.Hour}'])
        x.append(featureDic[f'entity_cnt_{item.entity_cnt}'])   # 共现实体数（加权）
        
        gap = int(abs(int(item.logTs)-firstTime)/1000) # 距离第一篇文章时间间隔（秒）
        if gap <= 599:
            x.append(featureDic[f'timeGap_{gap}s'])
        else:
            x.append(featureDic['timeGap_MoreThan10min'])
        x.append(featureDic['[sep]'])
        
    '''
    历史特征 (只添加一次)
    '''
    seqArr = group_df['seq'].values[0]
    if len(seqArr) == 1:  # 无历史信息
        x.append(featureDic['[noHistory]'])
        x.append(featureDic['[sep]'])
        continue
    seqArr = json.loads(seqArr.replace('\'', '\"'))
    assert len(seqArr)%2 == 0
    historyPair = []
    for i in range(0,len(seqArr),2):
        time = int(seqArr[i+1])
        itemId = int(seqArr[i])
        historyPair.append([itemId, time])
    historyPair = sorted(historyPair,key=lambda x:-x[1]) # 按时间排序离当前时间近的排前面
#         pprint(historyPair)
    for itemId, time in historyPair:
        historyTimeGap = int((firstTime - time)/1000/3600)
        if historyTimeGap<0:
            key = 'timeGap_LessThan0h'
        elif historyTimeGap>=100:
            key = 'timeGap_MoreThan100h'
        else:
            key = f'timeGap_{historyTimeGap}h'

        x.append(featureDic[f'itemId_{item.itemId}'])
        x.append(featureDic[key])
        x.append(featureDic['[sep]'])
#     print(len(x), x)
#     print(len(y), y)
    lengths.append(len(x))
    test_df.append([x])
#     break
test_df = pd.DataFrame(test_df)
lengths = pd.Series(lengths)
lengths.describe()

100%|██████████| 100112/100112 [03:31<00:00, 474.03it/s]


count    82145.000000
mean       131.383821
std         22.481205
min         22.000000
25%        120.000000
50%        144.000000
75%        144.000000
max       1329.000000
dtype: float64

In [114]:
test_df.columns = ['ids']
test_df.to_csv('./data/transformer_test_data.csv',index=False)
test_df

,ids
0,"[0, 720, 779, 1085, 1111, 0, 3344, 1101, 1136,..."
1,"[0, 720, 814, 1083, 1113, 0, 2139, 1090, 1132,..."
2,"[0, 737, 800, 1083, 1111, 0, 2893, 1090, 1137,..."
3,"[0, 738, 805, 1083, 1111, 0, 2139, 1090, 1132,..."
4,"[0, 720, 779, 1083, 1111, 0, 2139, 1090, 1136,..."
...,...
82140,"[0, 734, 787, 1083, 1111, 0, 4292, 1110, 1134,..."
82141,"[0, 711, 747, 1083, 1111, 0, 3784, 1110, 1133,..."
82142,"[0, 720, 757, 1083, 1111, 0, 4194, 1110, 1136,..."
82143,"[0, 724, 902, 1083, 1111, 0, 4339, 1110, 1132,..."


### 模型定义

#### sublayer

In [2]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        
        self.eps = eps
    
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into N heads
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * N * sl * d_model
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        

        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        output = self.out(concat)
    
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
    
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

#### layer

In [3]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model, dropout=dropout)
        self.ff = FeedForward(d_model, dropout=dropout)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    

#### embed

In [4]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 200, dropout = 0.1):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        pe = Variable(self.pe[:,:seq_len], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        return self.dropout(x)

#### model

In [11]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads, dropout):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model, max_seq_len=CFG.maxLen, dropout=dropout)
        self.layers = get_clones(EncoderLayer(d_model, heads, dropout), N)
        self.norm = Norm(d_model)
        self.linear = nn.Linear(d_model, 5) # 0,1(点击目标)  2(用户特征)  3(历史特征) 4(pad符号)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        output = self.linear(self.norm(x))
        return output

### 训练

#### 参数

In [12]:
class CFG:
    maxLen = 300
    embedDim = 128
    layers = 6
    heads = 8
    
    lr = 1e-4
    epoch = 10
    batchSize = 64
    dropout = 0.1
    numWorkers = 4
    device = '0'
    
    split = 0.95
    saved_path = './tmp'
    
# os.environ["CUDA_VISIBLE_DEVICES"] = CFG.device
# CFG.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CFG.device = torch.device("cpu")

if not os.path.exists(CFG.saved_path):
    os.mkdir(CFG.saved_path)

In [13]:
with open('./data/feature/featureDic.json') as f:
    vocab = json.load(f)
print('词表大小',len(vocab))
train_df = pd.read_csv('./data/transformer_train_data.csv')
test_df = pd.read_csv('./data/transformer_test_data.csv')
train_df

词表大小 350310


,ids,labels
0,"[0, 711, 747, 1083, 1130, 0, 2863, 1103, 1132,...","[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
1,"[0, 710, 746, 1083, 1111, 0, 1156, 1087, 1132,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 711, 747, 1083, 1112, 0, 1171, 1088, 1136,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 1, 1, 1, 1, ..."
3,"[0, 711, 747, 1083, 1111, 0, 2103, 1098, 1135,...","[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
4,"[0, 711, 747, 1083, 1111, 0, 1194, 1090, 1133,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
247077,"[0, 717, 785, 1083, 1111, 0, 2139, 1110, 1132,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
247078,"[0, 723, 992, 1083, 1111, 0, 3435, 1110, 1132,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
247079,"[0, 711, 747, 1083, 1125, 0, 2139, 1110, 1132,...","[2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
247080,"[0, 720, 757, 1083, 1111, 0, 1187, 1107, 1133,...","[2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
class TrainDataset(Dataset):
    def __init__(self, df, vocab, maxLen):
        self.ids = df['ids'].values
        self.labels = df['labels'].values
        self.maxLen = maxLen
        self.vocab = vocab
        
    def __len__(self):
        return len(self.ids)
    
    def prepare_input(self, ids, labels):
        assert len(ids) == len(labels)
        if len(ids)>=self.maxLen:
            ids = ids[:self.maxLen-1] + [vocab['[sep]']]
            labels = labels[:self.maxLen-1] + [2]
            masks = [1] * self.maxLen
        else:
            padLen = self.maxLen - len(ids)
            ids = ids + [vocab['[sep]']] * padLen
            labels = labels + [4] * padLen   # label pad = 4
            masks = [1] * (self.maxLen-padLen)+ [0] * padLen
#         print(len(ids) , len(labels) ,len(masks))
        assert len(ids) == len(labels) ==len(masks)
        ids = torch.LongTensor(ids)
        masks = torch.BoolTensor(masks)
        labels = torch.LongTensor(labels)
        return ids,masks,labels

    def __getitem__(self, idx):
        temp_id = json.loads(self.ids[idx])
        temp_label = json.loads(self.labels[idx])
        ids,masks,labels = self.prepare_input(temp_id, temp_label)
        
        return ids,masks,labels

class TestDataset(Dataset):
    def __init__(self, df, vocab, maxLen):
        self.ids = df['ids'].values
        self.maxLen = maxLen
        self.vocab = vocab
        
    def __len__(self):
        return len(self.ids)
    
    def prepare_input(self, ids, labels):
        if len(ids)>=self.maxLen:
            ids = ids[:self.maxLen-1] + [vocab['[sep]']]
            masks = [1] * self.maxLen
        else:
            padLen = self.maxLen - len(ids)
            ids = ids + [vocab['[sep]']] * padLen
            masks = [1] * len(ids) + [0] * padLen
        assert len(ids) == len(labels) ==len(masks)
        ids = torch.LongTensor(ids)
        masks = torch.BoolTensor(masks)
        return ids,masks

    def __getitem__(self, idx):
        ids,masks = self.prepare_input(self.ids[idx], self.labels[idx])
        return ids,masks

In [15]:
train = train_df.sample(frac=CFG.split) #按0.6比例随机采样
valid=train_df[~train_df.index.isin(train.index)]

train_dataset = TrainDataset(train, vocab, CFG.maxLen)
valid_dataset = TrainDataset(valid, vocab, CFG.maxLen)
test_dataset = TestDataset(test_df, vocab, CFG.maxLen)
print('数据量:', len(train_dataset), len(valid_dataset), len(test_dataset))

train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batchSize,
                              shuffle=True,
                              num_workers=CFG.numWorkers, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batchSize,
                              shuffle=True,
                              num_workers=CFG.numWorkers, pin_memory=True, drop_last=True)
test_loader = DataLoader(test_dataset,
                          batch_size=CFG.batchSize,
                          shuffle=False,
                          num_workers=CFG.numWorkers, pin_memory=True, drop_last=False)
print('batch数:', len(train_loader), len(valid_loader), len(test_loader))

数据量: 234728 12354 82145
batch数: 3667 193 1284


#### 主程序

In [17]:
model = Encoder(len(vocab), CFG.embedDim, CFG.layers, CFG.heads, CFG.dropout)
# model = model.cuda(device=CFG.device)
optimizer = torch.optim.Adam(model.parameters(), lr=CFG.lr, betas=(0.9, 0.98), eps=1e-9)
loss_fnc = nn.CrossEntropyLoss(weight=torch.from_numpy(np.array([1, 5, 0.5, 0.5,0.1])).float() ,
                                        size_average=True)#.cuda()
 # 0,1(点击目标)  2(用户特征)  3(历史特征) 4(pad符号)
def eval(model):
    model.eval()
    x, y = [], []
    for step, (ids, masks, labels) in enumerate(valid_loader):
        ids = ids#.cuda(device=CFG.device)
        masks = masks.unsqueeze(-2)#.cuda(device=CFG.device)
        with torch.no_grad():
            logits = model(ids, masks)
        batch_pred = logits.detach().cpu().numpy()
        labels = labels.cpu().numpy()
        ids = ids.cpu().numpy()
        
        for idx, pred_logits in enumerate(batch_pred):
            label = labels[idx]    
#             print(np.sum(label==0), np.sum(label==1))
            for i in range(2):
                ind = np.where(label==i)
                logits = pred_logits[ind]   # [标签数, 4] 
                logits = logits[:,:2 ]      # [标签数, 2] 
                logits = softmax(logits, axis=-1).argmax(-1)
                if len(logits):
                    truth = label[ind]
#                     x.extend(logits[:,1:].ravel().tolist())
                    x.extend(logits.ravel().tolist())
#                     print('111111', i, np.sum(truth==0), np.sum(truth==1), truth)
                    y.extend(truth.tolist())
#         sys.exit()
    print(f'truth为1个数={np.sum(np.array(y)==1)}\tpred为1个数={np.sum(np.array(x)==1)}')
    print(f'truth为0个数={np.sum(np.array(y)==0)}\tpred为1个数={np.sum(np.array(x)==0)}')
    return roc_auc_score(y,x), f1_score(y,x)
        
best_score = 0
step_num = len(train_loader)
for epoch in range(5):
    totol_loss = 0
    print('[Training] EPOCH = {:02d} '.format(epoch))
    tk0=tqdm(enumerate(train_loader), total=step_num)
    for i, (ids, masks, labels) in tk0:
        model.train()
        ids = ids#.cuda(device=CFG.device)
        masks = masks.unsqueeze(-2)#.cuda(device=CFG.device)
        labels = labels#.cuda(device=CFG.device)
        logits = model(ids, masks) # [batch_size, embed_dim, 4]
        loss = loss_fnc(logits.permute(0, 2, 1), labels)
        loss.backward()
        optimizer.step()
        totol_loss += loss.item()
        
        if (i+1) % int(step_num/4) == 0:
            auc,f1 = eval(model)
            print('[EVAL] epoch={:02d}\tstep={:03d}\taucScore={:.3f}\tf1Score={:.3f}'.format(epoch,i,auc,f1))
            if auc > best_score:
                best_score = auc
                torch.save(model, os.path.join(CFG.saved_path,'model.pth'))
                print('best score:', best_score)
#             print('epoch={:02d}\tstep={:03d}\tloss={:.3f}'.format(epoch,i,loss.item()))
        tk0.set_postfix(Epoch=epoch, Step=i, Loss=totol_loss/(i+1))
#         break
#     break    

[Training] EPOCH = 00 


  0%|          | 0/3667 [00:31<?, ?it/s]


KeyboardInterrupt: 

#### 输出结果